# Object Detection API Demo

<table align="left"><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

Welcome to the [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

> **Important**: This tutorial is to help you through the first step towards using [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to build models. If you just just need an off the shelf model that does the job, see the [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

In [1]:
!pip install -U --pre tensorflow=="2.*"

Requirement already up-to-date: tensorflow==2.* in c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages (2.0.0)


Make sure you have `pycocotools` installed

In [2]:
!pip install pycocotools

ERROR: Could not find a version that satisfies the requirement pycocotools (from versions: none)
ERROR: No matching distribution found for pycocotools


Get `tensorflow/models` or `cd` to parent directory of the repository.

In [1]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

'git' is not recognized as an internal or external command,
operable program or batch file.


Compile protobufs and install the object_detection package

In [2]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

Couldn't find program: 'bash'


In [3]:
%%bash 
cd models/research
pip install .

Couldn't find program: 'bash'


### Imports

In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from collections import Counter

Import the object detection module.

In [5]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import visualization_utils1 as vis_util1

Patches:

In [6]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [7]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [8]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = r'F:\models-master\research\object_detection\data\mscoco_label_map.pbtxt'
#PATH_TO_LABELS = r'F:\models-master\research\object_detection\data\mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

For the sake of simplicity we will test on 2 images:

In [9]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path(r'F:\cup and bottle')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('F:/cup and bottle/a.jpg'),
 WindowsPath('F:/cup and bottle/cup_img.jpg')]

# Detection

Load an object detection model:

In [10]:
model_name =  'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [11]:
#model_dir = 

In [12]:
#model = tf.saved_model.load(r"F:\faster_rcnn_resnet101_kitti_2017_11_08\saved_model")
#detection_model = model.signatures['serving_default']

Check the model's input signature, it expects a batch of 3-color images of type uint8: 

In [13]:
print(detection_model.inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


And retuns several outputs:

In [14]:
detection_model.output_dtypes

{'detection_scores': tf.float32,
 'detection_classes': tf.float32,
 'num_detections': tf.float32,
 'detection_boxes': tf.float32}

In [15]:
detection_model.output_shapes

{'detection_scores': TensorShape([None, 100]),
 'detection_classes': TensorShape([None, 100]),
 'num_detections': TensorShape([None]),
 'detection_boxes': TensorShape([None, 100, 4])}

Add a wrapper function to call the model, and cleanup the outputs:

In [18]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Run it on each test image and show the results:

In [16]:
import cv2

In [22]:
'''
lgo_img =  cv2.imread('logo.jpeg',cv2.IMREAD_UNCHANGED)
scl = 20
w = int(lgo_img.shape[1] * scl / 100)
h = int(lgo_img.shape[0] * scl / 100)
dim = (w,h)
lgo = cv2.resize(lgo_img, dim, interpolation = cv2.INTER_AREA)
lH,lW = lgo.shape[:2]
'''

"\nlgo_img =  cv2.imread('logo.jpeg',cv2.IMREAD_UNCHANGED)\nscl = 20\nw = int(lgo_img.shape[1] * scl / 100)\nh = int(lgo_img.shape[0] * scl / 100)\ndim = (w,h)\nlgo = cv2.resize(lgo_img, dim, interpolation = cv2.INTER_AREA)\nlH,lW = lgo.shape[:2]\n"

In [18]:
src = np.float32([[394, 538], [41, 292], [285, 14], [425, 41]])
dst = np.float32([[188, 486], [212, 298], [714, 317], [849, 475]])
hexa, status = cv2.findHomography(src,dst)

In [30]:
#secimg = cv2.imread(r'F:\googlemap\MSIL\90\cam16.jpg')
#secimg = cv2.resize(secimg,(540,544))
#cv2.imwrite(r'F:\googlemap\MSIL\90\as353.jpg',secimg)

In [19]:
import cv2
import pygame
import time
import math
cap = cv2.VideoCapture(r"F:\googlemap\MSIL\birdeyetool\cam15_Trim.mp4")
width = int(cap.get(3))
height = int(cap.get(4))
pygame.init()
screen = pygame.display.set_mode([960,540])
background = pygame.image.load(r'F:\googlemap\MSIL\birdeyetool\crop_destj.jpg')
running = True

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [21]:

while True and running:
    ret, image_np = cap.read()
    image_np = cv2.resize(image_np,(480,540))
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    #image = np.asarray(image_np)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
    output_dict = detection_model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
      # Visualization of the results of a detection.
    image,boxes,classname,classes = vis_util1.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      min_score_thresh=0.3,
      line_thickness=2)
    
    width = 480
    height = 540
    #print(boxes)
    center = []
    new_box= []
    for j in boxes:
        left =  int(j[1]*width)
        bottom = int(j[2]*height)
        right =int(j[3]*width)
        top = int(j[0]*height)
        val = (left,right,top, bottom)
        
        x = int((left+right)/2.0)
        y = int((top+bottom)/2.0)
        centr = (x,y)
        new_box.append(val)
        center.append(centr)

    #print(new_box)
    social_distance_violations = []
    distance_dict = {}
    for i in range(len(center)):
        for j in range(1, len(center)):
            d = (center[i][0]-center[j][0])**2 + (center[i][1]-center[j][1])**2
            dis = float(math.sqrt(d)) 
            if dis < 50 and dis > 0:
               # print("distance", dis)
                distance_dict[(new_box[i],new_box[j])] = dis
                social_distance_violations.append(new_box[i])
                social_distance_violations.append(new_box[j])
        
    social_distance_violations = list(set(social_distance_violations))
    #print(social_distance_violations)        
        

    for i in range(len(social_distance_violations)):
        start_point = (social_distance_violations[i][0], social_distance_violations[i][2])
        end_point = (social_distance_violations[i][1], social_distance_violations[i][3]) 
        color = (0, 0, 255)
        thickness = 3
        image = cv2.rectangle(image_np, start_point, end_point, color, thickness)        
        
        
        
    new_match = []
    for i in social_distance_violations:
        match = []
        for j in distance_dict:
                            
            if i in j:
                o,k = j
                if len(new_match)>0:
                    n = 0
                    for ii in new_match:
                        if o in ii or k in ii:
                            n+=1

                                                
                    if n == 0:
                        match.append(o)
                        match.append(k)
                                            
                else:
                    match.append(o)
                    match.append(k)

        if len(match)>0:
            match = list(set(match))

            new_match.append(match)


    #print(new_match)

   # print(len(new_match))
                

    points =[]
    for i in new_match:
        temp =[]
        for j in i:
            x,y,w,h = j
            centr_x = (x+y)/2
            centr_y = (w+h)/2

            center_ = [centr_x,centr_y]
            temp.append(center_)
        points.append(temp)

    #print(points)

    newcentroid = []
    for i in points:
        X = []
        Y = []
        for j in i:
            x,y = j
            X.append(x)
            Y.append(y)
        a = sum(X)/len(X)
        b = sum(Y)/len(Y)
        newcentroid.append((int(a),int(b)))
    #print("newcentroid",newcentroid)

                    
    for i in newcentroid:
        image = cv2.circle(image_np, i, 75, (255,0,0), 3)

    groups = "violated group {}".format(len(new_match))
    violated = 'Socialdistance violation {}'.format(len(social_distance_violations))
    cv2.rectangle(image_np,(30,35),(50,50),(0, 0, 255),-1)
    cv2.putText(image_np,violated ,(70,50), cv2.FONT_HERSHEY_SIMPLEX ,0.8, (0, 0, 255) ,1,cv2.LINE_AA)
    cv2.putText(image_np,groups ,(70,80), cv2.FONT_HERSHEY_SIMPLEX ,0.8, (0, 0, 255) ,1,cv2.LINE_AA)
   # image_np[475:475+lgo.shape[0], 10:10+lgo.shape[1]] = lgo   
        
        

        




    social_distance_violations_center = []
    for i in range(len(center)):
        for j in range(1, len(center)):
            d = (center[i][0]-center[j][0])**2 + (center[i][1]-center[j][1])**2
            dis = float(math.sqrt(d)) 
            if dis < 50 and dis > 0:
                print("distance", dis)
                X_i,Y_i = center[i]
                X_j,Y_j = center[j]
                xi = (int(X_i/4),int(Y_i/4))
                yi = (int(X_j/4),int(Y_j/4))
                social_distance_violations_center.append(center[i])
                social_distance_violations_center.append(center[j])
    
    social_distance_violations_center = list(set(social_distance_violations_center))
    #print(social_distance_violations_center)
    
    #screen.fill((255, 255, 255))
    

        # Draw a solid blue circle in the center
    screen.fill(pygame.Color("white"))
    screen.blit(background,(0,0))
    for i in center:
       # x,y = i
      #  x= int(x/4)
      #  y = int(y/4)
       # i= (x,y)
        if i in social_distance_violations_center:
            i = list(i)
            #print(i)
            a = np.array([i], dtype='float32')
            a = np.array([a])
            pointsOut = cv2.perspectiveTransform(a, hexa)
            pointsout = pointsOut[0][0]
            point_x = int(pointsout[0])
            point_y =  int(pointsout[1])
            
            circle_point = (point_x,point_y)
            pygame.draw.circle(screen, (255, 0, 0), circle_point, 10)
        else:
            i = list(i)
            a = np.array([i], dtype='float32')
            a = np.array([a])
            pointsOut = cv2.perspectiveTransform(a, hexa)
            pointsout = pointsOut[0][0]
            point_x = int(pointsout[0])
            point_y = int(pointsout[1])
            
            circle_point = (point_x,point_y)

            pygame.draw.circle(screen, (0, 0, 255), circle_point, 10)


        # Flip the display
        pygame.display.flip()
            #time.sleep(2)
        
    cv2.imshow('object detection',image_np)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break       
    
        
        

pygame.quit()

distance 47.4236228055175
distance 6.324555320336759
distance 7.280109889280518
distance 9.055385138137417
distance 10.04987562112089
distance 10.04987562112089
distance 11.045361017187261
distance 14.035668847618199
distance 13.0
distance 17.0
distance 18.027756377319946
distance 41.048751503547585
distance 27.202941017470888
distance 27.202941017470888
distance 18.027756377319946
distance 41.048751503547585
distance 27.202941017470888
distance 27.202941017470888
distance 17.0
distance 21.02379604162864
distance 23.08679276123039
distance 25.179356624028344
distance 25.079872407968907
distance 25.079872407968907
distance 25.019992006393608
distance 24.08318915758459
distance 25.079872407968907
distance 26.076809620810597
distance 41.182520563948
distance 24.166091947189145
distance 24.166091947189145
distance 27.018512172212592
distance 42.941821107167776
distance 23.706539182259394
distance 23.706539182259394
distance 27.018512172212592
distance 42.941821107167776
distance 23.7065391

In [25]:
.
v+

SyntaxError: invalid syntax (<ipython-input-25-c2ca2dec680a>, line 1)